In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path_to_data = '/kaggle/input/doom-crossing/'
path_to_doom = '/kaggle/input/doom-crossing/doom/'
path_to_animal = '/kaggle/input/doom-crossing/animal_crossing/'

img_dirs = []
for entry in os.scandir(path_to_data):
    if entry.is_dir():
        img_dirs.append(entry.path)
img_dirs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
image = plt.imread('/kaggle/input/doom-crossing/animal_crossing/5fqz3vkvw8q41.jpg')
plt.imshow(image)

In [ ]:
doom_dict = {}
for img_dir in img_dirs:
    doom_name = img_dir.split('/')[-1]
    file_list = []
    for entry in os.scandir(img_dir):
        file_list.append(entry.path)
    doom_dict[doom_name] = file_list
doom_dict

In [ ]:
class_dict = {}
count = 0
for doom_name in doom_dict.keys():
    class_dict[doom_name] = count
    count = count + 1
class_dict

In [ ]:
import cv2
import pywt

In [ ]:
img = cv2.imread('/kaggle/input/doom-crossing/animal_crossing/3xfc3u4woq051.jpg')
scalled_img = cv2.resize(img, (200, 200))
plt.imshow(scalled_img)

In [ ]:
def w2d(img, mode=None, level=1):
    imArray = img
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    imArray =  np.float32(imArray)   
    imArray /= 255;

    coeffs=pywt.wavedec2(imArray, mode, level=level)

    coeffs_H=list(coeffs)  
    coeffs_H[0] *= 0;  
    
    imArray_H=pywt.waverec2(coeffs_H, mode);
    imArray_H *= 255;
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [ ]:
X, y = [], []
for doom_name, training_files in doom_dict.items():
    for training_image in training_files:
        img = cv2.imread(training_image)
        scalled_raw_img = cv2.resize(img, (200, 200))
        img_har = w2d(img,'db1',5)
        scalled_img_har = cv2.resize(img_har, (200, 200))
        combined_img = np.vstack((scalled_raw_img.reshape(200*200*3,1),scalled_img_har.reshape(200*200,1)))
        X.append(combined_img)
        y.append(class_dict[doom_name])

In [ ]:
len(X[0])

In [ ]:
X = np.array(X).reshape(len(X),len(X[0])).astype(float)
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train.reshape(len(X_train), 200, 200, 4)
X_test = X_test.reshape(len(X_test), 200, 200, 4)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras

In [ ]:
model = Sequential()

#-- 1 layer -- 
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(200, 200, 4)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
#-- 2 layer -- 
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
#-- 3 layer -- 
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#-- 4 layer -- 
model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=32, epochs=5, shuffle=True)

In [ ]:
X_test

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
X_train = X_train.reshape(len(X_train), 200*200*4)
X_test = X_test.reshape(len(X_test), 200*200*4)

In [ ]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

For further optimization you can tune hyperparameters 